# 코랩 용 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !gdown https://drive.google.com/u/0/uc?id=1AWOO1awvSGHHKbydWJTmeZ0g5f5rV85I&export=download
# !unzip open.zip
!unzip open/train.zip 
!unzip open/test.zip
!mv train ./open
!mv test ./open 

In [ ]:
import os 
from glob import glob 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn 
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import transforms as T 
import cv2 
import torch.nn.functional as F
from tqdm import tqdm 

In [ ]:
# dir_default = './data/open/'
#코랩용 
dir_default = './open'

def file_load(dir_default):
    dir_train_imgs = sorted(glob(os.path.join(dir_default,'train','*')))
    dir_test_imgs = sorted(glob(os.path.join(dir_default,'test','*')))
    train_df = pd.read_csv(os.path.join(dir_default,'train_df.csv'))
    test_df = pd.read_csv(os.path.join(dir_default,'test_df.csv'))
    return dir_train_imgs,dir_test_imgs,train_df, test_df 

In [ ]:
dir_default = './open'
dir_train_imgs,dir_test_imgs,train_df, test_df  = file_load(dir_default)

#라벨 생성 
train_labels = train_df['label'].to_numpy()
#라벨 인코더 
label_encoder = {b:a for a,b in enumerate(np.unique(train_labels))}
label_decoder = {b:a for a,b in enumerate(np.unique(train_labels))}

# 데이터셋 

In [ ]:
class Dset(Dataset):
    def __init__(self,dir_imgs,labels):
        self.dir_imgs = dir_imgs 
        self.labels = labels 
        self.transform = self.transform_img()

    def __len__(self):
        return len(self.dir_imgs)

    def transform_img(self,train=True):
        if train:
            transform = T.Compose([
                T.ToTensor(),
                T.Resize(256),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                    )
            ])
        return transform 

    def load_img(self,dir_img):
        temp_img = cv2.imread(dir_img)
        temp_img = self.transform(temp_img)
        return temp_img 

    def __getitem__(self,idx):
        img = self.load_img(self.dir_imgs[idx])
        label = label_encoder[self.labels[idx]]
        return img, label 

# 모델 구성 

In [ ]:
class clf(nn.Module):
    def __init__(self):
        super(clf, self).__init__()
        self.layers = nn.Sequential(
            self.conv_block(3,4),
            self.conv_block(4,16),
            self.conv_block(16,64),
            self.conv_block(64,128),
            self.conv_block(128,256)
        )


    def conv_block(self,in_c,out_c):
      block = nn.Sequential(
          nn.Conv2d(in_c,out_c,3,padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.BatchNorm2d(out_c)
      )
      return block 
      
    def forward(self,x):
      x = self.layers(x)
      x = nn.Flatten()(x)
      x = nn.Linear(x.shape[-1],88)(x)
      x = F.softmax(x)
      return x

model = clf().to('cuda')

# 학습 

In [ ]:
# 데이터 로드 
dir_default = './open'
dir_train_imgs,dir_test_imgs,train_df, test_df  = file_load(dir_default)

#라벨 생성 
train_labels = train_df['label'].to_numpy()
#라벨 인코더 
label_encoder = {b:a for a,b in enumerate(np.unique(train_labels))}
label_decoder = {b:a for a,b in enumerate(np.unique(train_labels))}

#데이터 셋 
train_dset = Dset(dir_train_imgs,train_labels)
train_loader = DataLoader(train_dset,batch_size=16,shuffle=True)

#모델 및 컴파일 
device = 'cuda'
model = clf().to(device) 
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
criterion = nn.CrossEntropyLoss()


In [ ]:

for epoch in range(3):
  train_loss = 0.0 

  for i, data in tqdm(enumerate(train_loader)):
    img,labels = data 
    img = img.to(device)
    labels = labels.to(device)
    
    #옵티마이저 초기화 
    optimizer.zero_grad() 

    #backward 
    outputs = model(img)

    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item() 
  print(f'Epoch: {epoch}, train_loss : {train_loss}')    

0it [00:00, ?it/s]


RuntimeError: ignored

In [ ]:
!git push 